### **ETL - Accidentes Aéreos**

In [1]:
# Importación de librerías
import pandas as pd
import numpy as np

In [2]:
# Cargamos el archivo CSV en un DataFrame
df = pd.read_csv('Accidentes.csv')

In [3]:
# Convertimos la columna 'fecha' al formato de fecha
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')
# Creamos una nueva columna 'Año' que contiene el año de la fecha
df['Año'] = df['fecha'].dt.year

In [4]:
#Comprobando el año mayor
df['Año'].max()

2021

In [5]:
# Reemplazamos '?' por NaN en todo el DataFrame
df.replace('?', np.nan, inplace=True)

In [6]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Año
0,0,1908-09-17,1718,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",1908
1,1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,1909
2,2,1912-07-12,0630,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,1912
3,3,1913-08-06,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,1913
4,4,1913-09-09,1830,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20,NaN,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,1913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,"Near Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...,2021
5004,5004,2021-05-21,1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi...",2021
5005,5005,2021-06-10,0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...,2021
5006,5006,2021-07-04,11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",2021


In [7]:
# Convertimos la columna 'HORA declarada' en formato string para su posterior modificación
df['HORA declarada'] = df['HORA declarada'].astype(str)

In [8]:
# Definimo una función para limpiar y homogeneizar el formato de hora
def clean_and_homogenize_hour(hour_str):
    # Remueve 'c', 'c:', y 'Z' de la cadena
    cleaned_hour_str = hour_str.replace('c', '').replace('c:', '').replace('Z', '').strip()
    
    # Si el valor es un número de 4 dígitos, conviértelo a HHMM
    if len(cleaned_hour_str) == 4 and cleaned_hour_str.isdigit():
        return f"{cleaned_hour_str[:2]}:{cleaned_hour_str[2:]}:00"  # Agregar :00 para los segundos
    # Si el valor está en otros formatos, intenta ajustarlo a HH:MM:SS
    elif ':' in cleaned_hour_str:
        parts = cleaned_hour_str.split(':')
        if len(parts) == 2:
            return f"{parts[0].zfill(2)}:{parts[1].zfill(2)}:00"  # Agregar :00 para los segundos
    
    # Si no se puede convertir, devuelve NaN
    return pd.NaT

In [9]:
# Aplicamos la función de limpieza y homogeneización a la columna 'HORA declarada' y crea una nueva columna 'Hora'
df['Hora Accidente'] = df['HORA declarada'].apply(clean_and_homogenize_hour)

In [10]:
# Convertimos la columna 'Hora Accidente' a formato de tiempo
df['Hora Accidente'] = pd.to_datetime(df['Hora Accidente'], format='%H:%M:%S', errors='coerce').dt.time

In [11]:
# Comprobando el cambio a formato hora
df['Hora Accidente']

0       17:18:00
1            NaT
2       06:30:00
3            NaT
4       18:30:00
          ...   
5003    18:35:00
5004    18:00:00
5005    08:00:00
5006    11:30:00
5007    15:00:00
Name: Hora Accidente, Length: 5008, dtype: object

In [12]:
# Definimos una función para agrupar los años en décadas en orden
def categorize_decade(year):
    if 1901 <= year <= 1910:
        return "1901-1910"
    elif 1911 <= year <= 1920:
        return "1911-1920"
    elif 1921 <= year <= 1930:
        return "1921-1930"
    elif 1931 <= year <= 1940:
        return "1931-1940"
    elif 1941 <= year <= 1950:
        return "1941-1950"
    elif 1951 <= year <= 1960:
        return "1951-1960"
    elif 1961 <= year <= 1970:
        return "1961-1970"
    elif 1971 <= year <= 1980:
        return "1971-1980"
    elif 1981 <= year <= 1990:
        return "1981-1990"
    elif 1991 <= year <= 2000:
        return "1991-2000"
    elif 2001 <= year <= 2010:
        return "2001-2010"
    elif 2011 <= year <= 2020:
        return "2011-2020"
    elif 2021 <= year:
        return f"{year}-{year+9}"

In [13]:
# Aplicamos la función para crear la columna 'Decada'
df['Decada'] = df['Año'].apply(categorize_decade)

In [14]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,...,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Año,Hora Accidente,Decada
0,0,1908-09-17,1718,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,...,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",1908,17:18:00,1901-1910
1,1,1909-09-07,nan,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,...,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,1909,NaT,1901-1910
2,2,1912-07-12,0630,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,...,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,1912,06:30:00,1911-1920
3,3,1913-08-06,nan,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,...,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,1913,NaT,1911-1920
4,4,1913-09-09,1830,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,...,NaN,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,1913,18:30:00,1911-1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,"Near Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,...,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...,2021,18:35:00,2021-2030
5004,5004,2021-05-21,1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,...,7,4,11,7,4,0,"While on final approach, in poor weather condi...",2021,18:00:00,2021-2030
5005,5005,2021-06-10,0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,...,12,2,12,11,1,0,The plane was carrying military personnel and ...,2021,08:00:00,2021-2030
5006,5006,2021-07-04,11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,...,88,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",2021,11:30:00,2021-2030


In [19]:
# Obteniendo las décadas únicas en orden
decadas_unicas = df['Decada'].unique()
decadas_unicas.sort()

# Creando un diccionario para mapear las décadas a valores numéricos secuenciales
decada_a_valor = {decada: i + 1 for i, decada in enumerate(decadas_unicas)}

# Aplicamos el mapeo para crear la columna 'Valor Decada'
df['Valor Decada'] = df['Decada'].map(decada_a_valor)

In [20]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,...,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Año,Hora Accidente,Decada,Valor Decada
0,0,1908-09-17,1718,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,...,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",1908,17:18:00,1901-1910,1
1,1,1909-09-07,nan,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,...,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,1909,NaT,1901-1910,1
2,2,1912-07-12,0630,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,...,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,1912,06:30:00,1911-1920,2
3,3,1913-08-06,nan,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,...,1,1,0,1,0,The first fatal airplane accident in Canada oc...,1913,NaT,1911-1920,2
4,4,1913-09-09,1830,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,...,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,1913,18:30:00,1911-1920,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,"Near Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,...,1,5,4,1,0,The sightseeing helicopter crashed after missi...,2021,18:35:00,2021-2030,13
5004,5004,2021-05-21,1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,...,4,11,7,4,0,"While on final approach, in poor weather condi...",2021,18:00:00,2021-2030,13
5005,5005,2021-06-10,0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,...,2,12,11,1,0,The plane was carrying military personnel and ...,2021,08:00:00,2021-2030,13
5006,5006,2021-07-04,11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,...,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",2021,11:30:00,2021-2030,13


In [21]:
# Reemplazamos todos los caracteres que no sean letras o números en la columna 'Ruta'
df['Ruta'] = df['Ruta'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

# Reemplazamos todos los caracteres que no sean letras o números en la columna 'OperadOR'
df['OperadOR'] = df['OperadOR'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

In [22]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,...,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Año,Hora Accidente,Decada,Valor Decada
0,0,1908-09-17,1718,Fort Myer Virginia,Military US Army,NaN,Demonstration,Wright Flyer III,NaN,1,...,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",1908,17:18:00,1901-1910,1
1,1,1909-09-07,nan,JuvisysurOrge France,NaN,NaN,Air show,Wright Byplane,SC1,NaN,...,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,1909,NaT,1901-1910,1
2,2,1912-07-12,0630,Atlantic City New Jersey,Military US Navy,NaN,Test flight,Dirigible,NaN,NaN,...,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,1912,06:30:00,1911-1920,2
3,3,1913-08-06,nan,Victoria British Columbia Canada,Private,NaN,NaN,Curtiss seaplane,NaN,NaN,...,1,1,0,1,0,The first fatal airplane accident in Canada oc...,1913,NaT,1911-1920,2
4,4,1913-09-09,1830,Over the North Sea,Military German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,...,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,1913,18:30:00,1911-1920,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,Near Butte Alaska,Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,...,1,5,4,1,0,The sightseeing helicopter crashed after missi...,2021,18:35:00,2021-2030,13
5004,5004,2021-05-21,1800,Near Kaduna Nigeria,Military Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,...,4,11,7,4,0,"While on final approach, in poor weather condi...",2021,18:00:00,2021-2030,13
5005,5005,2021-06-10,0800,Near Pyin Oo Lwin Myanmar,Military Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,...,2,12,11,1,0,The plane was carrying military personnel and ...,2021,08:00:00,2021-2030,13
5006,5006,2021-07-04,11:30,Patikul Sulu Philippines,Military Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,...,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",2021,11:30:00,2021-2030,13


In [54]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,...,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary,Año,Hora Accidente,Decada,Valor Decada
0,0,1908-09-17,1718,Fort Myer Virginia,Military US Army,NaN,Demonstration,Wright Flyer III,NaN,1,...,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",1908,17:18:00,1901-1910,1
1,1,1909-09-07,nan,JuvisysurOrge France,NaN,NaN,Air show,Wright Byplane,SC1,NaN,...,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,1909,NaT,1901-1910,1
2,2,1912-07-12,0630,Atlantic City New Jersey,Military US Navy,NaN,Test flight,Dirigible,NaN,NaN,...,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,1912,06:30:00,1911-1920,2
3,3,1913-08-06,nan,Victoria British Columbia Canada,Private,NaN,NaN,Curtiss seaplane,NaN,NaN,...,1,1,0,1,0,The first fatal airplane accident in Canada oc...,1913,NaT,1911-1920,2
4,4,1913-09-09,1830,Over the North Sea,Military German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,...,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,1913,18:30:00,1911-1920,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,Near Butte Alaska,Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,...,1,5,4,1,0,The sightseeing helicopter crashed after missi...,2021,18:35:00,2021-2030,13
5004,5004,2021-05-21,1800,Near Kaduna Nigeria,Military Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,...,4,11,7,4,0,"While on final approach, in poor weather condi...",2021,18:00:00,2021-2030,13
5005,5005,2021-06-10,0800,Near Pyin Oo Lwin Myanmar,Military Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,...,2,12,11,1,0,The plane was carrying military personnel and ...,2021,08:00:00,2021-2030,13
5006,5006,2021-07-04,11:30,Patikul Sulu Philippines,Military Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,...,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",2021,11:30:00,2021-2030,13


In [55]:
# Función para asignar valores a Military y Comercial
def asignar_valores(row):
    if isinstance(row, str) and 'Military' in row:
        return 1, 0
    elif isinstance(row, str):
        return 0, 1
    else:
        return np.nan, np.nan

In [56]:
# Aplicamos la función a la columna 'OperadOR'
df['Military'], df['Comercial'] = zip(*df['OperadOR'].apply(asignar_valores))

In [57]:
# Comprobamos el DataFrame resultante
print(df[['OperadOR', 'Military', 'Comercial']])

                            OperadOR  Military  Comercial
0                  Military  US Army       1.0        0.0
1                                NaN       NaN        NaN
2                  Military  US Navy       1.0        0.0
3                            Private       0.0        1.0
4              Military  German Navy       1.0        0.0
...                              ...       ...        ...
5003               Soloy Helicopters       0.0        1.0
5004    Military  Nigerian Air Force       1.0        0.0
5005     Military  Myanmar Air Force       1.0        0.0
5006  Military  Philippine Air Force       1.0        0.0
5007   Kamchatka Aviation Enterprise       0.0        1.0

[5008 rows x 3 columns]


In [67]:
# Convertimos las columnas 'all_aboard' y 'cantidad_de_fallecidos' a números (si es posible)
df['all_aboard'] = pd.to_numeric(df['all_aboard'], errors='coerce')
df['cantidad de fallecidos'] = pd.to_numeric(df['cantidad de fallecidos'], errors='coerce')

In [68]:
# Función para calcular la columna 'Sobrevivientes'
def calcular_sobrevivientes(row):
    all_aboard = row['all_aboard']
    cantidad_de_fallecidos = row['cantidad de fallecidos']

    if pd.isna(all_aboard):
        return np.nan
    elif pd.isna(cantidad_de_fallecidos):
        return all_aboard
    elif cantidad_de_fallecidos > all_aboard:
        return 0
    else:
        return all_aboard - cantidad_de_fallecidos

In [69]:
# Aplicamos la función a las filas del DataFrame
df['Sobrevivientes'] = df.apply(calcular_sobrevivientes, axis=1)

In [70]:
df

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,...,crew_fatalities,ground,summary,Año,Hora Accidente,Decada,Valor Decada,Military,Comercial,Sobrevivientes
0,0,1908-09-17,1718,Fort Myer Virginia,Military US Army,NaN,Demonstration,Wright Flyer III,NaN,1,...,0,0,"During a demonstration flight, a U.S. Army fly...",1908,17:18:00,1901-1910,1,1.0,0.0,1.0
1,1,1909-09-07,nan,JuvisysurOrge France,NaN,NaN,Air show,Wright Byplane,SC1,NaN,...,0,0,Eugene Lefebvre was the first pilot to ever be...,1909,NaT,1901-1910,1,NaN,NaN,0.0
2,2,1912-07-12,0630,Atlantic City New Jersey,Military US Navy,NaN,Test flight,Dirigible,NaN,NaN,...,5,0,First U.S. dirigible Akron exploded just offsh...,1912,06:30:00,1911-1920,2,1.0,0.0,0.0
3,3,1913-08-06,nan,Victoria British Columbia Canada,Private,NaN,NaN,Curtiss seaplane,NaN,NaN,...,1,0,The first fatal airplane accident in Canada oc...,1913,NaT,1911-1920,2,0.0,1.0,0.0
4,4,1913-09-09,1830,Over the North Sea,Military German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,...,NaN,0,The airship flew into a thunderstorm and encou...,1913,18:30:00,1911-1920,2,1.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,2021-03-28,1835,Near Butte Alaska,Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,...,1,0,The sightseeing helicopter crashed after missi...,2021,18:35:00,2021-2030,13,0.0,1.0,1.0
5004,5004,2021-05-21,1800,Near Kaduna Nigeria,Military Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,...,4,0,"While on final approach, in poor weather condi...",2021,18:00:00,2021-2030,13,1.0,0.0,0.0
5005,5005,2021-06-10,0800,Near Pyin Oo Lwin Myanmar,Military Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,...,1,0,The plane was carrying military personnel and ...,2021,08:00:00,2021-2030,13,1.0,0.0,2.0
5006,5006,2021-07-04,11:30,Patikul Sulu Philippines,Military Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,...,NaN,3,"While attempting to land at Jolo Airport, the ...",2021,11:30:00,2021-2030,13,1.0,0.0,46.0


In [75]:
# Convertir la columna  a formato de números
df['passenger_fatalities'] = pd.to_numeric(df['passenger_fatalities'], errors='coerce')

In [83]:
# Función para calcular el número de sobrevivientes pasajeros
def calcular_sobrevivientes_pasajeros(row):
    cantidad_de_fallecidos = row['cantidad de fallecidos']
    passenger_fatalities = row['passenger_fatalities']

    if pd.isna(cantidad_de_fallecidos) or pd.isna(passenger_fatalities):
        return np.nan
    elif cantidad_de_fallecidos < passenger_fatalities:
        return 0
    else:
        return cantidad_de_fallecidos - passenger_fatalities

In [84]:
# Aplicamos la función a las filas del DataFrame
df['Sobrevivientes_Pasajeros'] = df.apply(calcular_sobrevivientes_pasajeros, axis=1)

In [85]:
# Guardamos en un csv para el posterior EDA
df.to_csv('Acci.csv', index=False)